# Import des Librairies / Fonctions

In [15]:
import pandas as pd
import math
from thefuzz import fuzz
from unidecode import unidecode

# Fonction calcul de la distance

In [16]:
# Fonction haversine pour calculer la distance entre deux points géographiques
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    R = 6371000  # Rayon de la Terre en mètres
    distance = R * c
    return distance

# Lecture des fichier d'entré

In [17]:
# Fonction pour charger les fichiers CSV avec les types de données spécifiés
def load_csv_with_dtypes(filepath, dtype_dict):
    return pd.read_csv(filepath, delimiter=';', encoding='utf-8', low_memory=False, dtype=dtype_dict)

# Lire les fichiers CSV en sautant la première ligne (indice 0) et en utilisant ";" comme séparateur
# Spécifiez les dtypes des colonnes si nécessaire
dtype_g2gai = {
    'commune_lgt': str,
    'nmr_lgt': str
}
dtype_gmbi = {
    'nbPiecesPpales': str,
    'situationOccup': str,
    'typePersonne_1': str,
    'siren_1': str,
    'formJuridiq_1': str,
    'siren_2': str,
    'formJuridiq_2': str,
    'typePersonne_2': str
}

# Chargement des fichiers CSV
df1 = load_csv_with_dtypes('G2GAI_unused Lacalisation.csv', dtype_g2gai)
df2 = load_csv_with_dtypes('GMBI_unused Localisation.csv', dtype_gmbi)


# Initialisation des listes / variables

In [18]:
results = []  # Initialiser une liste pour stocker les résultats
used_indices_gmbi = set()  # Set pour garder la trace des indices des lignes déjà appariées
used_indices_g2gai = set()  # Set pour garder la trace des indices des lignes déjà appariées

date_fin = "31/12/2023"
date_debut = "01/01/2024"

max_ecart_etage = 5
max_ecart_pieces = 5
max_ecart_surface = 40

# Recherche des correspondances

In [19]:
# Itérer à travers chaque ligne du premier fichier (df1)
for i in range(len(df1)):
    lat1 = df1.iloc[i, 38]
    lon1 = df1.iloc[i, 39]
    dept1 = df1.iloc[i, 2]
    # commune1 = unidecode(df1.iloc[i, 10])
    commune1 = df1.iloc[i, 10]
    if pd.notna(commune1) and isinstance(commune1, str):
        commune1 = unidecode(commune1)
    else:
        commune1 = ''
    if df1.iloc[i,55] == "ok":
        commune_validation1 = unidecode(df1.iloc[i, 49])
    else:
        commune_validation1 = "nul"
    g2gai_logement = df1.iloc[i, 12]
    nom_courant = df1.iloc[i, 27]
    prenom_courant = df1.iloc[i, 28]
    # Vérifier si prenom_courant est pésent pour extraire que le premier prénom
    if isinstance(prenom_courant, str) and prenom_courant.strip() != "":
        prenom_courant = prenom_courant.split(' ')[0]
    else:
        prenom_courant = ""
    date_naissance_courant = df1.iloc[i, 29]
    etage_g2gai = df1.iloc[i, 11]
    pieces_g2gai = df1.iloc[i, 14]
    surface_g2gai = df1.iloc[i, 13]

    # Vérifier si le libellé de la commune correspond à l'information de la colonne d'indice 49
    if fuzz.partial_token_sort_ratio(commune1, commune_validation1) != 100:
        continue

    # Initialiser la distance minimale comme infinie et la correspondance correspondante comme None
    min_distance = float('inf')
    best_match_index = None

    best_indices = []  # Liste pour stocker les indices des meilleures correspondances

    # Itérer à travers chaque ligne du deuxième fichier (df2)
    for j in range(len(df2)):
        # Vérifier si la ligne n'a pas déjà été utilisée et si la cellule en indice 84 est "ok" et si la cellule en indice 18 contient "partie"
        if j not in used_indices_gmbi and df2.iloc[j, 84] == "ok" and pd.notna(df2.iloc[j, 18]) and "partie" in unidecode(df2.iloc[j, 18]).lower():
            dept2 = df2.iloc[j, 4]
            commune2 = unidecode(df2.iloc[j, 6])
            commune_validation2 = unidecode(df2.iloc[j, 78])
            etage_gmbi = df2.iloc[j, 12]
            pieces_gmbi = df2.iloc[j, 20]
            surface_gmbi = df2.iloc[j, 21]

            # Vérifier si le libellé de la commune correspond à l'information de la colonne d'indice 78
            if fuzz.partial_token_sort_ratio(commune1, commune_validation2) != 100:
                continue

            # Vérifier si les départements et les communes correspondent
            if dept1 == dept2 and fuzz.partial_token_sort_ratio(commune1, commune2) == 100:
                lat2 = df2.iloc[j, 67]
                lon2 = df2.iloc[j, 68]

                # Calculer la distance
                distance = haversine(lon1, lat1, lon2, lat2)

                # Vérifier si la distance est inférieure à 500 mètres
                if distance < 500:
                    # Vérifier si les étages, nombre de pièces et surface sont dans les écarts maximaux
                    if abs(etage_g2gai - etage_gmbi) <= max_ecart_etage and \
                    abs(int(pieces_g2gai) - int(pieces_gmbi)) <= max_ecart_pieces and \
                    abs(surface_g2gai - surface_gmbi) <= max_ecart_surface:
                        
                        print(distance)
                        
                        # Mettre à jour la liste des meilleurs indices si la distance est la même que min_distance
                        if distance == min_distance:
                            best_indices.append(j)
                        # Si la distance est plus petite que min_distance, mettre à jour
                        elif distance < min_distance:
                            min_distance = distance
                            best_indices = [j]

    # Maintenant, parcourir les meilleures correspondances pour trouver celle qui se rapproche le plus de surface_g2gai
    if best_indices:
        best_match_index = best_indices[0]
        best_match_surface_diff = abs(surface_g2gai - df2.iloc[best_match_index, 21])
        for idx in best_indices:
            surface_diff = abs(surface_g2gai - df2.iloc[idx, 21])
            if surface_diff < best_match_surface_diff:
                best_match_index = idx
                best_match_surface_diff = surface_diff


    # Ajouter la meilleure correspondance pour la ligne actuelle de df1 aux résultats
    if best_match_index is not None:

        used_indices_g2gai.add(i)  # Ajouter l'indice à l'ensemble des indices utilisés

        best_match = df2.iloc[best_match_index].copy()
        decla = df2.iloc[best_match_index].copy()

        # if best_match.iloc[33] == "2":
        if best_match.iloc[45] == "01/01/2024":
            best_match.iloc[1] = 'X'
            best_match.iloc[64] = g2gai_logement
            best_match = best_match.iloc[:67]  # Garder seulement les colonnes jusqu'à l'indice 67
            results.append(best_match)
            used_indices_gmbi.add(best_match_index)  # Ajouter l'indice à l'ensemble des indices utilisés
            # Nouvelle déclaration
            decla.iloc[0] = 'decla'  # Mettre "X" dans la cellule d'indice 0
            decla.iloc[35] = nom_courant
            decla.iloc[37] = prenom_courant
            decla.iloc[38] = date_naissance_courant
            if pd.notna(decla.iloc[35]):
                decla.iloc[32] = "4"
                decla.iloc[33] = "1"
            else:
                decla.iloc[32] = "3"
                decla.iloc[33] = ""
            decla.iloc[39:44] = ''
            decla.iloc[45] = date_debut
            decla.iloc[64] = g2gai_logement
            decla.iloc[47:64] = ''
            decla = decla.iloc[:67]  # Garder seulement les colonnes jusqu'à l'indice 67
            results.append(decla)

        elif best_match.iloc[33] == "2" or best_match.iloc[33] == "1":
            best_match.iloc[0] = 'X'
            best_match.iloc[32] = "4"
            best_match.iloc[46] = date_fin
            if pd.notna(best_match.iloc[47]):
                best_match.iloc[60] = date_fin
            best_match.iloc[64] = g2gai_logement
            best_match = best_match.iloc[:67]  # Garder seulement les colonnes jusqu'à l'indice 67
            results.append(best_match)
            used_indices_gmbi.add(best_match_index)  # Ajouter l'indice à l'ensemble des indices utilisés
            # Nouvelle déclaration
            decla.iloc[0] = 'decla'  # Mettre "X" dans la cellule d'indice 0
            decla.iloc[35] = nom_courant
            decla.iloc[37] = prenom_courant
            decla.iloc[38] = date_naissance_courant
            if pd.notna(decla.iloc[35]):
                decla.iloc[32] = "4"
                decla.iloc[33] = "1"
            else:
                decla.iloc[32] = "3"
                decla.iloc[33] = ""
            decla.iloc[45] = date_debut
            decla.iloc[64] = g2gai_logement
            decla.iloc[47:64] = ''
            decla = decla.iloc[:67]  # Garder seulement les colonnes jusqu'à l'indice 67
            results.append(decla)

        else:
            # Déclaration direct
            decla.iloc[0] = 'test'  # Mettre "X" dans la cellule d'indice 0
            decla.iloc[35] = nom_courant
            decla.iloc[37] = prenom_courant
            decla.iloc[38] = date_naissance_courant
            if pd.notna(decla.iloc[35]):
                decla.iloc[32] = "4"
                decla.iloc[33] = "1"
            else:
                decla.iloc[32] = "3"
                decla.iloc[33] = ""
            decla.iloc[45] = date_debut
            decla.iloc[64] = g2gai_logement
            decla.iloc[47:64] = ''
            decla = decla.iloc[:67]  # Garder seulement les colonnes jusqu'à l'indice 67
            results.append(decla)

        # Rechercher les locaux associé au logement
        num_fiscal_pp = best_match.iloc[2]
        for k in range(len(df2)):
            if k not in used_indices_gmbi and df2.iloc[k,3] == num_fiscal_pp:
                match = df2.iloc[k].copy()
                decla_log_associe = df2.iloc[k].copy()

                # if match.iloc[33] == "2":
                if match.iloc[45] == "01/01/2024":
                    match.iloc[1] = 'X'
                    match.iloc[64] = g2gai_logement
                    match = match.iloc[:67]  # Garder seulement les colonnes jusqu'à l'indice 67
                    results.append(match)
                    used_indices_gmbi.add(k)  # Ajouter l'indice à l'ensemble des indices utilisés
                    # Nouvelle déclaration
                    decla_log_associe.iloc[0] = 'decla'  # Mettre "X" dans la cellule d'indice 0
                    decla_log_associe.iloc[32] = "4"
                    decla_log_associe.iloc[35] = nom_courant
                    decla_log_associe.iloc[37] = prenom_courant
                    decla_log_associe.iloc[38] = date_naissance_courant
                    if pd.notna(decla_log_associe.iloc[35]):
                        decla_log_associe.iloc[32] = "4"
                        decla_log_associe.iloc[33] = "1"
                    else:
                        decla_log_associe.iloc[32] = "3"
                        decla_log_associe.iloc[33] = ""
                    decla_log_associe.iloc[39:44] = ''
                    decla_log_associe.iloc[45] = date_debut
                    decla_log_associe.iloc[64] = g2gai_logement
                    decla_log_associe.iloc[47:64] = ''
                    decla_log_associe = decla_log_associe.iloc[:67]  # Garder seulement les colonnes jusqu'à l'indice 67
                    results.append(decla_log_associe)

                elif match.iloc[33] == "2" or match.iloc[33] == "1":
                    match.iloc[0] = 'X'  # Mettre "X" dans la cellule d'indice 0
                    match.iloc[32] = "4"
                    match.iloc[46] = date_fin
                    if pd.notna(match.iloc[47]):
                        match.iloc[60] = date_fin
                    match.iloc[64] = g2gai_logement
                    match = match.iloc[:67]  # Garder seulement les colonnes jusqu'à l'indice 67
                    results.append(match)
                    used_indices_gmbi.add(k)  # Ajouter l'indice à l'ensemble des indices utilisés
                    # Nouvelle déclaration
                    decla_log_associe.iloc[0] = 'decla'  # Mettre "X" dans la cellule d'indice 0
                    decla_log_associe.iloc[32] = "4"
                    decla_log_associe.iloc[35] = nom_courant
                    decla_log_associe.iloc[37] = prenom_courant
                    decla_log_associe.iloc[38] = date_naissance_courant
                    if pd.notna(decla_log_associe.iloc[35]):
                        decla_log_associe.iloc[32] = "4"
                        decla_log_associe.iloc[33] = "1"
                    else:
                        decla_log_associe.iloc[32] = "3"
                        decla_log_associe.iloc[33] = ""
                    decla_log_associe.iloc[45] = date_debut
                    decla_log_associe.iloc[64] = g2gai_logement
                    decla_log_associe.iloc[47:64] = ''
                    decla_log_associe = decla_log_associe.iloc[:67]  # Garder seulement les colonnes jusqu'à l'indice 67
                    results.append(decla_log_associe)

                else:
                    # Nouvelle déclaration
                    decla_log_associe.iloc[0] = 'test'  # Mettre "X" dans la cellule d'indice 0
                    decla_log_associe.iloc[32] = "4"
                    decla_log_associe.iloc[35] = nom_courant
                    decla_log_associe.iloc[37] = prenom_courant
                    decla_log_associe.iloc[38] = date_naissance_courant
                    if pd.notna(decla_log_associe.iloc[35]):
                        decla_log_associe.iloc[32] = "4"
                        decla_log_associe.iloc[33] = "1"
                    else:
                        decla_log_associe.iloc[32] = "3"
                        decla_log_associe.iloc[33] = ""
                    decla_log_associe.iloc[45] = date_debut
                    decla_log_associe.iloc[64] = g2gai_logement
                    decla_log_associe.iloc[47:64] = ''
                    decla_log_associe = decla_log_associe.iloc[:67]  # Garder seulement les colonnes jusqu'à l'indice 67
                    results.append(decla_log_associe)

# Écriture des résultats dans les fichier de sortie

In [ ]:
# Convertir la liste de résultats en DataFrame
result_df = pd.DataFrame(results)

# Afficher le DataFrame résultant
print(result_df)

# Enregistrer le DataFrame résultant dans un nouveau fichier CSV
result_df.to_csv('rendu TEST.csv', index=False, sep=';', encoding='utf-8')

# Filtrer les lignes non utilisées de G2GAI
df1_unused = df1[~df1.index.isin(used_indices_g2gai)]
# Écrire dans un nouveau fichier CSV
df1_unused.to_csv('G2GAI_unused TEST.csv', index=False, sep=';', encoding='utf-8')

# Filtrer les lignes non utilisées de GMBI
df2_unused = df2[~df2.index.isin(used_indices_gmbi)]
# Écrire dans un nouveau fichier CSV
df2_unused.to_csv('GMBI_unused TEST.csv', index=False, sep=';', encoding='utf-8')


     declarer  annuler_occupation noFiscalDuLocal     idGroupLoc cdDept  \
38          X                 NaN   721540201158A            NaN     72   
38      decla                 NaN   721540201158A            NaN     72   
55          X                 NaN   721540500850N  721540201158A     72   
55      decla                 NaN   721540500850N  721540201158A     72   
41          X                 NaN   721540201154T            NaN     72   
...       ...                 ...             ...            ...    ...   
3945    decla                 NaN   976120028441Y            NaN    976   
3952        X                 NaN   976120028439V            NaN    976   
3952    decla                 NaN   976120028439V            NaN    976   
3947     test                 NaN   976120028442U            NaN    976   
3947     test                 NaN   976120028442U            NaN    976   

     libelle_departement libelle_commune  noVoirie  indRep     libelle_voie  \
38                SA